In [2]:
%use lets-plot, krangl, kotlin-statistics, klaxon
import java.time.LocalDate
import java.util.*
import java.time.ZoneId

In [37]:
import java.net.URL
fun DataFrame.Companion.fromJsonObject(url: String, mapper: (JsonObject) -> JsonArray<*>): DataFrame {
    val uri = URL(url).toURI().toURL()
    val obj = Parser.default().parse(uri.openStream()) as JsonObject
    val arr = obj.let(mapper).toJsonString()
    return DataFrame.fromJsonString(arr)
}

fun <R> DataFrame.mapStringColumn(column: String, default: R, mapFunc: (String) -> R?) =
    addColumn(column) { it[column].map<String> { i -> mapFunc(i) ?: default } }
infix fun DataCol.notEq(i: Any) = !eq(i)

In [42]:
val dataUrl = "https://data.taipei/api/v1/dataset/f4a75ba9-7721-4363-884d-c3820b0b917c?scope=resourceAquire"
val df = DataFrame.fromJsonObject(dataUrl) { it.obj("result")!!.array<Any?>("results")!! }
        .mapStringColumn("Bodyweight", 0.0) {it: String ->
            it.substringBefore("KG", "0").toDoubleOrNull()
        }

In [43]:
df.schema()

DataFrame with 365 observations
AcceptNum        [Str]  106060601, 106060401, 106060210, 106060207, 106060203, 106053107, 106052807, 106052604, 106052603, 1...
IsSterilization  [Str]  未絕育, 已絕育, 未絕育, 未絕育, 未絕育, 未絕育, 未絕育, 未絕育, 未絕育, 已絕育, 未絕育, 未絕育, 未絕育, 未絕育, 未絕育, 已絕育, 未絕育, 未絕育, 未絕育, 未絕育, ...
Name             [Str]  , , 俊俊, 小莊, 好橙, 歐雅, 青茶, 烏莉, 安亞, 吉好, 霖霖, 斑恩, 南風, 美荷, 董仔, 妍妍, 瓦諾, 琪妍, 亞當, 西蒙, 吉哥, 山迪, 小白, 泉泉, , 蘇珊, , ...
Variety          [Str]  米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, 米克斯, ...
Age              [Str]  成年, 成年, 成年, 成年, 成年, 年輕, 年輕, 成年, 年輕, 成年, 成年, 成年, 成年, 成年, 年輕, 成年, 成年, 成年, 成年, 成年, 成年, 成年, 成年, 成年, 成年, ...
ChildreAnlong    [Str]  , , , , , , , , , , , , , , , 不確定, , , , , , , , , , , , , , , , , , , , , , , , , , , 可, 可, 可, 可, 可...
Resettlement     [Str]  臺北市動物之家 收容編號106060601, , 臺北市動物之家 收容編號106060210, 臺北市動物之家 收容編號106060207, 臺北市動物之家 收容編號106060203, 臺北市動物之...
Sex              [Str]  雄, 雌, 雄, 雄, 雄, 雌, 雄, 雌, 雌, 雄, 雄, 雄, 雄, 雌, 雄, 雌, 

In [44]:
val dfPlots = lets_plot(df.toMap())

In [50]:
dfPlots + geom_bar() { x = "Type" } + ggtitle("收容動物種類統計")

In [51]:
dfPlots + geom_bar() { x = "Type"; fill = "Sex" } + ggtitle("收容動物種類統計(性別比)")

In [52]:
dfPlots + geom_bar() { x = "Type"; fill = "IsSterilization" } + ggtitle("收容動物種類統計(絕育比)")

In [55]:
dfPlots + geom_bar() { x = "Type"; fill = "Age" } + ggtitle("收容動物種類統計(年齡比)")

In [58]:
dfPlots + facet_grid("Type") + geom_bar { x = "Sex"; fill = "IsSterilization" } + ggtitle("性別對絕育的比例關係")

In [70]:
lets_plot(
    df.filter { it["Bodyweight"] notEq 0.0 }.toMap()
) + geom_boxplot { x = "Type"; y = "Bodyweight" } +
    geom_point { x = "Type"; y = "Bodyweight" } +
    ggtitle("體重分佈")